# Hint Strategy 1
This notebook demonstrates how an LLM performs when presented with a Javaee -> Quarkus JMS issue and provide a hint that can be used to improve the basic rule.
Installing pre requisites and configuring Kai with `GPT-4o-mini`

In [17]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

Found existing installation: kai 0.1.1.dev8+g923abc3
Uninstalling kai-0.1.1.dev8+g923abc3:
  Successfully uninstalled kai-0.1.1.dev8+g923abc3
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/konveyor/kai.git (to revision main) to /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-rhw45l2i
  Running command git clone --filter=blob:none --quiet https://github.com/konveyor/kai.git /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-rhw45l2i
  Resolved https://github.com/konveyor/kai.git to commit 923abc35262ed0e36094c91ee20f2e0cabc3bbf5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kai: filename=kai-0.1.1.dev8+g923abc3-py3-none-any.whl size=4435742 sha256=ecae9c70eb14e1e4533dc16cdcd6f926b10e615aaf4221c9851adf760f9882eb
  Stored in directory: /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/

In [18]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
load_dotenv() 

# Initialize the model provider using GPT-4o via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "gpt-3.5-turbo"},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



Below is the snippet we are trying to migrate from javaee to Qusrkus. This code uses JMS which needs to be migarted to Quarkus equivalent.
Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [19]:
before_code = """\
import javax.jms.* 

public class InventoryNotificationMDB implements MessageListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    private final static String JNDI_FACTORY = "weblogic.jndi.WLInitialContextFactory";
    private final static String JMS_FACTORY = "TCF";
    private final static String TOPIC = "topic/orders";
    private TopicConnection tcon;
    private TopicSession tsession;
    private TopicSubscriber tsubscriber;

    public void onMessage(Message rcvMessage) {
        TextMessage msg;
        {
            try {
                System.out.println("received message inventory");
                if (rcvMessage instanceof TextMessage) {
                    msg = (TextMessage) rcvMessage;
                    String orderStr = msg.getBody(String.class);
                    Order order = Transformers.jsonToOrder(orderStr);
                    order.getItemList().forEach(orderItem -> {
                        int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                        int new_quantity = old_quantity - orderItem.getQuantity();
                        if (new_quantity < LOW_THRESHOLD) {
                            System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                        } else {
                            orderItem.setQuantity(new_quantity);
                        }
                    });
                }


            } catch (JMSException jmse) {
                System.err.println("An exception occurred: " + jmse.getMessage());
            }
        }
    }
    }
"""

This rule flags suggests migrating javaee JMS referesnces with Quarkus specific ones.

In [20]:
hint_to_migrate="""\
    
  description:  References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.
  message: |
   References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.

    """

summary_of_changes="""\
Remove Deprecated APIs: Identify and remove any Java EE components and replace them with Quarkus or MicroProfile equivalents.
Use Annotations: Leverage Quarkus annotations like @Incoming, @ApplicationScoped, and @Transactional for better integration and functionality.
Logging: Utilize org.jboss.logging.Logger for logging instead of traditional Java logging or System.out.
Error Handling: Implement robust error handling with logging and exception management.
Testing: Ensure that the refactored code is thoroughly tested to confirm that it behaves as expected in the new environment.
"""

Similar example showing before and after migration

In [21]:
example_before_code="""\

@MessageDriven(
    activationConfig = {
        @ActivationConfigProperty(propertyName = "destinationLookup", propertyValue = "jms/queue/myQueue"), // JNDI lookup
        @ActivationConfigProperty(propertyName = "destinationType", propertyValue = "jakarta.jms.Queue")
        // Other properties like connectionFactoryLookup, acknowledgeMode might be needed depending on the server
    }
)
public class SimpleQueueMDB implements MessageListener {

    private static final Logger log = Logger.getLogger(SimpleQueueMDB.class.getName());

    @Override
    public void onMessage(Message message) {
        try {
            if (message instanceof TextMessage) {
                TextMessage textMessage = (TextMessage) message;
                String text = textMessage.getText();
                log.log(Level.INFO, "MDB Received: {0}", text);
               
                processPayload(text);
            } else {
                log.log(Level.WARNING, "MDB Received non-text message: {0}", message);
            }
            // Container typically acknowledges based on configuration (default AUTO_ACKNOWLEDGE implicit here)
        } catch (Exception e) {
            log.log(Level.SEVERE, "MDB Error processing message", e);

        }
    }
}

"""

example_after_code="""\
import jakarta.enterprise.context.ApplicationScoped;
import org.eclipse.microprofile.reactive.messaging.Incoming;
import org.jboss.logging.Logger;

@ApplicationScoped
public class SimpleQueueListener {

    private static final Logger log = Logger.getLogger(SimpleQueueListener.class);
    @Incoming("my-queue-in")
    public void consume(String payload) { 
        log.infof("Reactive Listener Received: %s", payload);
        try {

            processPayload(payload);

        } catch (Exception e) {
            log.errorf(e, "Reactive Listener Error processing message: %s", payload);
            throw new RuntimeException("Processing failed for: " + payload, e);
        }
    }

}

"""

In [22]:
source="javaee"
target="quarkus"

In [23]:
prompt = f"""

The following java code needs to be migrated from {source} to {target}.

```java
{before_code}
```
I will provide you with an example before/after code migration
```java
{example_before_code}
{example_after_code}
```
Use the following hint's message and summary of changes from a previous migration to migrate. 

{hint_to_migrate}
{summary_of_changes}

Also, remove the deprecated and unused APIs and suggest the correct alternatives based on the migration needs. 

Can you provide a refactored version of {before_code} that is compatible with {target}?
Also, create a step by step process for the above migration? Summarize and create a hint with the changes.

"""

In [24]:

response = await rendered_llm_call(prompt)

To migrate the given code from Java EE to Quarkus, we need to replace the JMS related imports and annotations with their Quarkus equivalents. Here is the refactored version of the code:

```java
import org.eclipse.microprofile.reactive.messaging.Incoming;
import org.jboss.logging.Logger;

public class InventoryNotificationMDB {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    public void onMessage(String orderStr) {
        try {
            System.out.println("received message inventory");
            Order order = Transformers.jsonToOrder(orderStr);
            order.getItemList().forEach(orderItem -> {
                int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                int new_quantity = old_quantity - orderItem.getQuantity();
                if (new_quantity < LOW_THRESHOLD) {
                    System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                } else {
                    orderItem.setQuantity(new_quantity);
                }
            });
        } catch (Exception e) {
            System.err.println("An exception occurred: " + e.getMessage());
        }
    }
}
```

Step by step process for the migration:

1. Replace the import statement `import javax.jms.*` with `import org.eclipse.microprofile.reactive.messaging.Incoming;` and `import org.jboss.logging.Logger;`.
2. Remove the `implements MessageListener` interface from the class declaration.
3. Remove the JMS related properties and variables like `JNDI_FACTORY`, `JMS_FACTORY`, `TOPIC`, `TopicConnection`, `TopicSession`, and `TopicSubscriber`.
4. Modify the `onMessage` method to accept a `String` parameter instead of a `Message` parameter.
5. Update the method logic to process the `orderStr` directly instead of extracting it from a `TextMessage`.
6. Handle exceptions using proper error handling techniques like logging.

Hint for the changes:
- Replace Java EE JMS imports with Quarkus MicroProfile Reactive Messaging imports.
- Remove JMS related properties and variables.
- Modify the method signature to accept a `String` parameter.
- Update the method logic to process the message directly.
- Implement proper error handling using logging.